In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [23]:
num_nodes = 4
dim = 100

In [24]:
embedding1 = nn.Embedding(num_nodes, dim)
embedding2 = nn.Embedding(num_nodes, dim)
linear1 = nn.Linear(dim, num_nodes)

linear2 = nn.Linear(dim, num_nodes)

In [60]:
_k = 2
_alpha = 10

In [61]:
nodevec1 = embedding1.weight
nodevec2 = embedding1.weight

In [62]:
nodevec1 = torch.tanh(_alpha * linear1(nodevec1))
nodevec2 = torch.tanh(_alpha * linear2(nodevec2))

In [63]:
nodevec1.shape

torch.Size([4, 4])

In [64]:
a = torch.mm(nodevec1, nodevec2.transpose(1, 0)) - torch.mm(
            nodevec2, nodevec1.transpose(1, 0)
        )

In [65]:
a.shape

torch.Size([4, 4])

In [66]:
a

tensor([[ 0.0000,  1.7620, -2.2454, -1.7549],
        [-1.7620,  0.0000, -3.6542,  0.2094],
        [ 2.2454,  3.6542,  0.0000, -3.9545],
        [ 1.7549, -0.2094,  3.9545,  0.0000]], grad_fn=<SubBackward0>)

In [67]:
A = F.relu(torch.tanh(_alpha * a))

In [68]:
A

tensor([[0.0000, 1.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.9701],
        [1.0000, 1.0000, 0.0000, 0.0000],
        [1.0000, 0.0000, 1.0000, 0.0000]], grad_fn=<ReluBackward0>)

In [69]:
mask = torch.zeros(num_nodes, num_nodes)

In [70]:
mask

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [71]:
mask.fill_(float("0"))

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [72]:
s1, t1 = A.topk(_k, 1)

In [73]:
s1

tensor([[1.0000, 0.0000],
        [0.9701, 0.0000],
        [1.0000, 1.0000],
        [1.0000, 1.0000]], grad_fn=<TopkBackward0>)

In [74]:
t1

tensor([[1, 0],
        [3, 1],
        [0, 1],
        [0, 2]])

In [75]:
mask.scatter_(1, t1, s1.fill_(1))

tensor([[1., 1., 0., 0.],
        [0., 1., 0., 1.],
        [1., 1., 0., 0.],
        [1., 0., 1., 0.]], grad_fn=<ScatterBackward0>)

In [76]:
A = A * mask

In [77]:
A

tensor([[0.0000, 1.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.9701],
        [1.0000, 1.0000, 0.0000, 0.0000],
        [1.0000, 0.0000, 1.0000, 0.0000]], grad_fn=<MulBackward0>)